In [3]:
import pandas as pd
import re
#from rapidfuzz import fuzz, process

In [2]:
import pandas as pd
import re
from rapidfuzz import fuzz, process

# Función de limpieza de texto para una columna específica del DataFrame
def clear_column(column):
    """
    Limpia y normaliza el texto en una columna para eliminar términos irrelevantes y uniformar el formato.
    """
    # Eliminamos palabras irrelevantes como unidades, envases, etc.
    columna_limpia = re.sub(
        r'\b(\d+|mg|comp\.?|blisters?|estuche|x|ml|unidades|marca|botella|caja|referencia|cm|g|m|f\.a\.x|'
        r'jbe\.|sol\.|env\.|lata|vial|pvo\.|sobres|gotero|cápsulas|frascos?|blister|bot\.|mcg|sp|p\.b\.)\b', 
        '', 
        column, 
        flags=re.IGNORECASE
    )
    # Reemplazar saltos de línea por espacios
    columna_limpia = re.sub(r'\n', ' ', columna_limpia)
    columna_limpia = re.sub(r'[:\.\-,;()]', '', columna_limpia)
    columna_limpia = re.sub(r'\s+', ' ', columna_limpia).strip()
    columna_limpia = columna_limpia.lower()
    # Eliminamos palabras comunes (como 'de', 'la', etc.) que no ayudan a la comparación
    columna_limpia = re.sub(r'\b(?:de|la|el|en|a|y|con|para|del|sobre|por|ac|se)\b', '', columna_limpia).strip()
    columna_limpia = re.sub(r'\b(\w+)\s+\1\b', r'\1', columna_limpia)
    # Aseguramos que unidades como mg, ml, etc. no interfieran
    columna_limpia = re.sub(r'\b(\d+mg|\d+g|\d+ml)\b', '', columna_limpia)
    return columna_limpia

# Función para cargar los datos desde un archivo CSV
def load_data(df_archivo):
    """
    Carga un archivo CSV en un DataFrame de pandas.
    """
    try:
        df = pd.read_csv(df_archivo)
        return df
    except Exception as e:
        print(f"Error al cargar el archivo {df_archivo}: {e}")
        return None

# Función para filtrar productos por laboratorio
def laboratorio_input_df(df, columna):
    """
    Busca coincidencias parciales en una columna de un DataFrame basado en una entrada del usuario.
    """
    input_name = input("Ingrese el nombre del laboratorio: ").strip()
    tablas_coincidencias = df[df[columna].str.contains(input_name, case=False, na=False)]

    if not tablas_coincidencias.empty:
        print(f"Se encontraron {len(tablas_coincidencias)} productos coincidentes del laboratorio '{input_name}'.")
        return tablas_coincidencias
    else:
        print(f"No se encontraron productos del laboratorio '{input_name}'.")
        return None

# Función para normalizar celdas con múltiples productos
def normalize_dataframe(df, column_to_split):
    """
    Normaliza un DataFrame separando celdas con múltiples productos en filas individuales.
    Args:
        df (pd.DataFrame): DataFrame original.
        column_to_split (str): Nombre de la columna a normalizar.
    Returns:
        pd.DataFrame: DataFrame normalizado con un producto por fila.
    """
    # Separar productos por múltiples delimitadores: saltos de línea, comas, puntos y comas, etc.
    df[column_to_split] = df[column_to_split].astype(str).str.split(r'[\n,;]+')  # Ahora cubre más delimitadores

    # Expandir las listas en múltiples filas
    df_normalized = df.explode(column_to_split, ignore_index=True)

    # Limpiar espacios extra en las celdas
    df_normalized[column_to_split] = df_normalized[column_to_split].str.strip()

    return df_normalized

# Función de comparación difusa
def comparar_similitud(texto1, texto2, umbral=80):
    """
    Compara dos textos utilizando similitud difusa (fuzzy matching) con un umbral configurable.
    """
    # Limpiar ambos textos antes de la comparación
    texto1_limpio = clear_column(texto1)
    texto2_limpio = clear_column(texto2)
    
    # Calcular la similitud entre los dos textos usando partial_ratio
    similitud = fuzz.partial_ratio(texto1_limpio, texto2_limpio)
    
    # Verificar si la similitud es mayor o igual al umbral
    return similitud >= umbral, similitud

# Cargar los datos desde archivos CSV
name_df_provedor = "prueba2.csv"  
name_df_productos = "L_STMaestroPrd.csv"  

df_proveedor = load_data(name_df_provedor)
df_productos = load_data(name_df_productos)

# Normalizar las columnas con múltiples productos
df_proveedor = normalize_dataframe(df_proveedor, 'DESCRIPCIÓN')
df_productos = normalize_dataframe(df_productos, 'descrip1')

# Copias de los DataFrames
copy_dfProductos = df_productos.loc[:, df_productos.notna().any()].copy()  # Eliminar columnas vacías
copy_dfProveedores = df_proveedor.copy()

# Limpiar las descripciones en ambas tablas
copy_dfProductos['descrip1_limpia'] = copy_dfProductos['descrip1'].fillna('').apply(clear_column)
copy_dfProveedores['descripcion_limpia'] = copy_dfProveedores['DESCRIPCIÓN'].fillna('').apply(clear_column)

# Filtrar los productos por laboratorio
columna_nombre = 'atrib0'  # Cambia por el nombre de la columna que contiene el laboratorio
laboratorio_coincidencias_df = laboratorio_input_df(copy_dfProductos, columna_nombre)

if laboratorio_coincidencias_df is not None:
    # Crear índice para búsqueda rápida
    proveedor_index = copy_dfProveedores['descripcion_limpia'].tolist()

    matches = []

    # Recorrer productos del proveedor y buscar todas las coincidencias con los productos del laboratorio
    for _, prov_row in copy_dfProveedores.iterrows():
        prov_desc = prov_row['descripcion_limpia']

        # Buscar todas las coincidencias que superen el umbral de similitud
        all_matches = process.extract(prov_desc, laboratorio_coincidencias_df['descrip1_limpia'], scorer=fuzz.partial_ratio, limit=None)
        valid_matches = [match for match in all_matches if match[1] >= 80]  # Filtrar por umbral de 80

        if valid_matches:
            for match in valid_matches:
                matched_row = laboratorio_coincidencias_df[ 
                    laboratorio_coincidencias_df['descrip1_limpia'] == match[0]
                ].iloc[0]  # Obtener la fila coincidente del laboratorio

                # Combinar la información del producto del proveedor y del laboratorio
                match_info = prov_row.to_dict()  # Información del proveedor
                match_info.update(matched_row.to_dict())  # Información del producto del laboratorio
                match_info['similaridad'] = match[1]  # Similaridad

                matches.append(match_info)
        else:
            # Si no hay coincidencia, incluir sólo la información del proveedor
            match_info = prov_row.to_dict()
            match_info['similaridad'] = 0  # Similaridad = 0

            # Agregar columnas vacías para los datos del laboratorio
            for col in laboratorio_coincidencias_df.columns:
                if col not in match_info:
                    match_info[col] = None

            matches.append(match_info)

    # Convertir la lista de matches a DataFrame
    df_matches = pd.DataFrame(matches)

    # Reordenar columnas para que 'similaridad' esté al principio
    cols = ['similaridad'] + [col for col in df_matches.columns if col != 'similaridad']
    df_matches = df_matches[cols]

    # Mostrar resultados
    print(f"\nCoincidencias encontradas: {len(df_matches[df_matches['similaridad'] > 0])}")
    print(df_matches)

    # Productos del laboratorio sin coincidencias
    productos_sin_coincidencia = laboratorio_coincidencias_df[~laboratorio_coincidencias_df['descrip1_limpia'].isin(
        df_matches[df_matches['similaridad'] > 0]['descrip1_limpia']
    )]

    print(f"\nProductos del laboratorio sin coincidencias: {len(productos_sin_coincidencia)}")
    print(productos_sin_coincidencia)
else:
    print("No se encontraron coincidencias para el laboratorio especificado.")


ModuleNotFoundError: No module named 'rapidfuzz'

In [8]:
df_matches

,similaridad,Unnamed: 0,DESCRIPCIÓN,PRESENTACION,CODIGO DE BARRAS,PRECIO CON IVA,PRECIO SIN IVA,PUBLICO\nSUGERIDO,descripcion_limpia,niprod,...,tipo_egreso,pc_diftrans,cod_cvta,cat_item,cant_rep,pto_rep,atrib0,atrib1,atrib2,descrip1_limpia
0,100.0,20015888\n20005289\n20021980,HUMIRA AC x 2 (20mg),"2 JERINGAS PRELL. X 0,2 ml\n2 LAPICERAS PRELL....",8054083017068\n8054083014050\n8054083017648,"$ 740,145.59\n$ 1,480,292....","$ 611,690.58\n$ 1,223,382...","$ 1,073,211.02\n$ 2,146,42...",humira,119550.0,...,E,0.0,NaN,ITGEV,0,0.0,ABBVIE,NaN,NaN,humira / lapprell
1,100.0,20015888\n20005289\n20021980,HUMIRA AC x 2 (20mg),"2 JERINGAS PRELL. X 0,2 ml\n2 LAPICERAS PRELL....",8054083017068\n8054083014050\n8054083017648,"$ 740,145.59\n$ 1,480,292....","$ 611,690.58\n$ 1,223,382...","$ 1,073,211.02\n$ 2,146,42...",humira,119551.0,...,E,0.0,NaN,ITGEV,0,0.0,ABBVIE,NaN,NaN,humira / jerprell
2,100.0,20015888\n20005289\n20021980,HUMIRA AC x 2 (20mg),"2 JERINGAS PRELL. X 0,2 ml\n2 LAPICERAS PRELL....",8054083017068\n8054083014050\n8054083017648,"$ 740,145.59\n$ 1,480,292....","$ 611,690.58\n$ 1,223,382...","$ 1,073,211.02\n$ 2,146,42...",humira,119550.0,...,E,0.0,NaN,ITGEV,0,0.0,ABBVIE,NaN,NaN,humira / lapprell
3,100.0,20015888\n20005289\n20021980,HUMIRA AC x 2 (20mg),"2 JERINGAS PRELL. X 0,2 ml\n2 LAPICERAS PRELL....",8054083017068\n8054083014050\n8054083017648,"$ 740,145.59\n$ 1,480,292....","$ 611,690.58\n$ 1,223,382...","$ 1,073,211.02\n$ 2,146,42...",humira,13684.0,...,E,0.0,NaN,ITGEV,0,0.0,CASSARÁ,NaN,NaN,ira
4,100.0,20015888\n20005289\n20021980,HUMIRA AC x 2 (20mg),"2 JERINGAS PRELL. X 0,2 ml\n2 LAPICERAS PRELL....",8054083017068\n8054083014050\n8054083017648,"$ 740,145.59\n$ 1,480,292....","$ 611,690.58\n$ 1,223,382...","$ 1,073,211.02\n$ 2,146,42...",humira,13684.0,...,E,0.0,NaN,ITGEV,0,0.0,CASSARÁ,NaN,NaN,ira
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,80.0,93348AR,OZURDEX,0.7 MG C/1,7795316000150,"$ 1.291.438,06","$ 1.067.304,19","$ 1.872.585,02",ozurdex,9872.0,...,E,0.0,NaN,ITGEV,0,0.0,ARISTON,NaN,NaN,ex3
352,80.0,93348AR,OZURDEX,0.7 MG C/1,7795316000150,"$ 1.291.438,06","$ 1.067.304,19","$ 1.872.585,02",ozurdex,9872.0,...,E,0.0,NaN,ITGEV,0,0.0,ARISTON,NaN,NaN,ex3
353,80.0,93348AR,OZURDEX,0.7 MG C/1,7795316000150,"$ 1.291.438,06","$ 1.067.304,19","$ 1.872.585,02",ozurdex,9872.0,...,E,0.0,NaN,ITGEV,0,0.0,ARISTON,NaN,NaN,ex3
354,80.0,93348AR,OZURDEX,0.7 MG C/1,7795316000150,"$ 1.291.438,06","$ 1.067.304,19","$ 1.872.585,02",ozurdex,9872.0,...,E,0.0,NaN,ITGEV,0,0.0,ARISTON,NaN,NaN,ex3


In [9]:
# Mostrar las coincidencias que no cumplen con el umbral de similitud
df_matches[df_matches['similaridad'] < 80]

,similaridad,Unnamed: 0,DESCRIPCIÓN,PRESENTACION,CODIGO DE BARRAS,PRECIO CON IVA,PRECIO SIN IVA,PUBLICO\nSUGERIDO,descripcion_limpia,niprod,...,tipo_egreso,pc_diftrans,cod_cvta,cat_item,cant_rep,pto_rep,atrib0,atrib1,atrib2,descrip1_limpia
21,0.0,20062091\n20069005\n20078725,RINVOQ 15MG RINVOQ 30MG,15 MG x 30 COMP REC DE LIB. PROLONGADA\n30 MG ...,8054083018706\n8054083021386\n8054083023922,"$ 1,799,606.71\n$ 3,184,521.02...","$ 1,487,278.28\n$ 2,631,835....","$ 2,609,429.58\n$ 4,617,55...",rinvoq rinvoq,NaN,...,None,NaN,NaN,None,None,NaN,None,NaN,NaN,None
